In [2]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [3]:
import os,sys
sys.path.append('./misc/lib/python3.7/site-packages')

import math
import numpy as np
import requests
import nglview as nv
import ipywidgets as widgets
%matplotlib notebook
import matplotlib.pyplot as plt
import IPython.display as Display
from IPython.display import display, display_markdown
from ipywidgets import Layout
from pathlib import Path

import parmed as pmd
import re

import hublib.use
from hublib.ui import FileUpload, Download
from hublib.cmd import runCommand

from scipy.ndimage import gaussian_filter

%use gromacs-5.1.4
%use pymol-1.8.4

np.set_printoptions(precision=8)
np.set_printoptions(suppress=True)


HTMLButtonPrompt = '''<html>
<head>
<meta name="viewport" content="width=device-width, initial-scale=1">
</head>
<body>
<a href="{link}" target="_blank" >
<button class="p-Widget jupyter-widgets jupyter-button widget-button mod-warning" style="width:150px; background-color:#CCCCCC; font-size:10pt; color:black">{text}</button>
</a>
</body>
</html>
'''   

# Width of elements in site parameter grid
grid_width = '2.5cm'
ggap = '0.5cm'
grid_total_width = '12cm'

<IPython.core.display.Javascript object>

In [6]:
#################################################################
# Model load dialogue
#################################################################

param_drop = widgets.Dropdown(
    options=[],
    description='',
    disabled=False,
    layout = Layout(width='5cm')
)

refresh_button = widgets.Button(
    description='Refresh List',
    layout=Layout(width='2.5cm')
)

load_button = widgets.Button(
    description='Load Model',
    layout=Layout(width='2.5cm')
)


def refresh_onclick(b):
    out = !{'ls params/*_names.txt'}
    prefList = []
    for line in out:
        if line[-4:]=='.txt':
            prefList.append('_'.join(line.split('_')[0:-1]).split('/')[-1])
    param_drop.options = prefList
    if len(prefList)>0:
        param_drop.value = prefList[0]

refresh_button.on_click(refresh_onclick)

def load_button_onclick(b):
    global Coups
    global Freqs
    global Dips
    global Names
    global ParamBox
    global FreqBoxList
    global ParamBoxList
    global FWHMBoxList
    
    # If no parameter files are available, abort
    if len(param_drop.options)==0:
        return 0
    
    prefix = param_drop.value
    
    Coups = np.loadtxt('params/' + prefix + '_coups.txt')
    Freqs = np.loadtxt('params/' + prefix + '_freqs.txt')
    Dips = np.loadtxt('params/' + prefix + '_dips.txt')
    with open('params/' + prefix + '_names.txt') as file:
        NameString = file.readline()
    Names = []
    for name in NameString.strip().split(" "):
        Names.append(name)
        
    # Reshape all variables. This ensures that the data types
    # are treated consistently, even if only a single oscillator
    # is present. 
    Nsites = len(Names)
    Coups.shape = (Nsites,Nsites)
    Freqs.shape = (Nsites,)
    Dips.shape = (Nsites,3)
        
    LblList = []
    FreqBoxList = []
    FWHMBoxList = []
    ParamBoxList = []
    for n in range(0, len(Freqs)):
        LblList.append(widgets.Label(value=Names[n], layout=Layout(width=grid_width)))
        
        FreqBoxList.append(
            widgets.BoundedFloatText(
                value=Freqs[n],
                disabled=False,
                min=0.0,
                max=20000.0,
                step = 1.0,
                layout=Layout(width=grid_width)
            ))

        FWHMBoxList.append(widgets.BoundedFloatText(
                value=0.0,
                min=0.0,
                max=2000.0,
                width='1cm',
                disabled=False, 
                step = 1.0,
                layout=Layout(width=grid_width)
            ))
    
    for n in range(0, len(Freqs)):
        ParamBoxList.append(widgets.Box([LblList[n], FreqBoxList[n], FWHMBoxList[n]], layout=Layout(flex='row', grid_gap=ggap)))
    ParamBox.children = [HeadBox] + ParamBoxList + [ParamDescript]
    calc_button.disabled = False
    specfile.value = param_drop.value
    
load_button.on_click(load_button_onclick)

refresh_onclick(0)
Coups = []
Freqs = []
Dips = []
Names = []



#################################################################
# Spectrum window setup
#################################################################


def format_coord(x, y):
    #return 'x=%.1f, y=%1.2f'%(x, y)
    return ''


def build_axes():
    specax.set_yticks([])
    specax.set_xlabel('$\\bar\\nu$ (cm$^{-1}$)')
    specax2.xaxis.tick_top()
    specax2.set_xlabel('$\\lambda$ (nm)')
    specax2.format_coord = format_coord
    update_ax2()
    plt.tight_layout()
    plt.show()

def update_ax2():
    specax2.set_xlim(specax.get_xlim())
    xticks2 = (1.0e7)/(specax.get_xticks())
    dlambda = -np.mean(np.diff(xticks2))
    ndec = max(-int(np.ceil(np.log10(dlambda))), 0)
    fmt = "{:." + str(ndec) + "f}"
    xticklabels2 = []
    for tick in xticks2:
        xticklabels2.append(fmt.format(tick))
    
    specax2.set_xticks((1.0e7)/xticks2)
    specax2.set_xticklabels(xticklabels2)
    specax2.set_xlim(specax.get_xlim())
    plt.tight_layout()
    
    
def update_spec_lims():
    
    lines = [abs_line]
    myinf = 1e+100
    mny = myinf
    mxy = -myinf
    for line in lines:
        if line.get_visible()==True:
            mny = min(mny, np.min(line.get_ydata()))
            mxy = max(mxy, np.max(line.get_ydata()))
    if (mny<myinf) and (mxy>-myinf) and (mny!=mxy):
        specax.set_ylim(mny, mxy)
        
    mnx = len(vaxis) - 1
    mxx = 0
    cutoff = 1e-6
    for line in lines:
        if line.get_visible()==True:
            ndcs = np.where(np.abs(line.get_ydata())>cutoff)[0]
            if len(ndcs>0):
                mnx = min(mnx, np.min(ndcs-1))
                mxx = max(mxx, np.max(ndcs+1))
    if mxx>mnx:
        specax.set_xlim(vaxis[mnx], vaxis[mxx])
        update_ax2()
    plt.tight_layout()

        
vaxis = np.arange(10000, 18000)

specout = widgets.Output(layout=Layout(width='4.25in',
                                      height='4.25in'))



#################################################################
# Spectrum calculation setup
#################################################################


def calculate_spec(b):
    global Freqs
    global progress_bar
    
    Nsites = len(Freqs)
    
    for n in range(0, Nsites):
        Freqs[n] = FreqBoxList[n].value
    
    Sigmas = np.zeros((Nsites,))
    if disorder_cb.value==True:
        Nreps = Nits
        for n in range(0, Nsites):
            Sigmas[n] = FWHMBoxList[n].value/2.355
    else:
        Nreps = int(1)
    
    Ham0 = Coups + np.diag(Freqs)
    
    Nplot = 1000
    SIGMA = 4
    progress_bar.max = Nreps
    abs_spec = np.zeros(np.shape(vaxis))
    for n in range(0, Nreps):
        dfvec = np.zeros((Nsites,))
        for p in range(0, Nsites):
            dfvec[p] = np.random.normal(0, Sigmas[p])
            
        Ham = Ham0 + np.diag(dfvec)
        eVals,eVecs = np.linalg.eigh(Ham)
        eMu = np.matmul(np.transpose(eVecs), Dips)

        for p in range(0,Nsites):
            ndx = np.where(np.abs(vaxis-eVals[p])==np.min(np.abs(vaxis-eVals[p])))[0][0]
            abs_spec[ndx] += np.linalg.norm(eMu[p])**2/Nsites #mu is a 3 element vector
        progress_bar.value = n + 1
        
        # Every Nplot cycles we update the plot
        if (n+1)%Nplot==0:
            abs_line.set_ydata(gaussian_filter(abs_spec, sigma=SIGMA))
            update_spec_lims()
            specfig.canvas.draw()
            specfig.canvas.flush_events()
            
    abs_spec = gaussian_filter(abs_spec, sigma=SIGMA)
    abs_line.set_ydata(abs_spec)
    
    update_spec_lims()
    


HeadBox = widgets.Box([
    #widgets.Label(value="Pigment:", layout=Layout(width='2cm')),
    widgets.HTML(value="<center><b style='font-size:12pt'>Pigment</b></center>", layout=Layout(width=grid_width)),
    widgets.HTML(value="<center><b style='font-size:12pt'>Frequency</b></center>", layout=Layout(width=grid_width)),
    widgets.HTML(value="<center><b style='font-size:12pt'>FHWM</b></center>", layout=Layout(width=grid_width)),
    ], layout=Layout(flex='row', grid_gap=ggap))

calc_button = widgets.Button(
    description='Calculate',
    disabled=True,
    tooltip='Click to calculate spectra',
)

calc_button.on_click(calculate_spec)

progress_bar = widgets.IntProgress(
    value=0,
    min=0,
    max=1,
    step=1,
    description='',
    bar_style='success', # 'success', 'info', 'warning', 'danger' or ''
    orientation='horizontal',
    layout=Layout(width='7cm')
)


nit_slider = widgets.FloatLogSlider(
    value=0,
    base=10,
    min=0, # max exponent of base
    max=5, # min exponent of base
    step=1, # exponent step
    description='Iterations',
    disabled=True,
    readout_format='d',
    layout=Layout(width='15cm',
                 margin='5pt')
)

Nits = int(round(nit_slider.value))
def nit_on_change(v):
    global Nits
    Nits = int(round(v['new']))
nit_slider.observe(nit_on_change, names='value')

disorder_cb = widgets.Checkbox(
    value=False,
    disabled=False,
    description='Disorder?',
    layout=Layout(width='2.5cm',
                 margin='5pt')
)
disorder_cb.style.description_width='0pt'

def dis_cb_onclick(b):
    nit_slider.disabled = (not disorder_cb.value)
disorder_cb.observe(dis_cb_onclick)

specfilelbl = widgets.Label(value='Output file prefix: ')

specfile = widgets.Text(
    value='test',
    placeholder='file prefix',
    layout = widgets.Layout(width='5cm'),
    disabled=False
)

export_button = widgets.Button(
    description='Save Spectrum',
    layout=Layout(width='3cm'))

def export_button_onclick(b):
    prefix = specfile.value
    np.savetxt('spec/' + prefix + '_abs.txt', np.vstack([vaxis, abs_line.get_ydata()]).T)
export_button.on_click(export_button_onclick)



LblList = []
FreqBoxList = []
FWHMBoxList = []
ParamBoxList = []

DisorderBox = widgets.Box([disorder_cb, nit_slider], 
                         layout=Layout(flex_flow='row',
                                      width=grid_total_width))




SetDescript = widgets.HTML('<b style="font-size:12pt">Set Defaults</b>')


def fwhm_onclick(b):
    global FWHMBoxList
    for widg in FWHMBoxList:
        widg.value = fwhm_value.value
    
fwhm_button = widgets.Button(
    description='Set all',
    layout=Layout(width='2cm')
)
fwhm_button.on_click(fwhm_onclick)

fwhm_value = widgets.BoundedFloatText(
    description='',
    value=200.0,
    min=0.0,
    step=1.0,
    max=2000.0,
    layout=Layout(width='2cm')
)

fwhm_lbl = widgets.Label(
    value='Default FWHM:',
    layout=Layout(width='2.5cm')
)

FWHMDefBox = widgets.Box([fwhm_lbl, fwhm_value, fwhm_button], 
                     layout=Layout(
                         flex_flow = 'row',
                         margin='0.25cm'))

def freq_onclick(b):
    global FreqBoxList
    for widg in FreqBoxList:
        widg.value = freq_value.value
    
freq_button = widgets.Button(
    description='Set all',
    layout=Layout(width='2cm')
)
freq_button.on_click(freq_onclick)

freq_value = widgets.BoundedFloatText(
    description='',
    value=14925.0,
    min=0.0,
    step=1.0,
    max=20000.0,
    layout=Layout(width='2cm')
)

freq_lbl = widgets.Label(
    value='Site Energy:',
    layout=Layout(width='2.5cm')
)

FreqDefBox = widgets.Box([freq_lbl, freq_value, freq_button], 
                     layout=Layout(
                         flex_flow = 'row',
                         margin='0.25cm'))

SetDefBox = widgets.Box([SetDescript, FreqDefBox, FWHMDefBox], 
                       layout=Layout(
                           flex_flow='column',
#                            margin='0.25cm',
#                            border='solid',
                           width=grid_total_width
                       ))
spec_down = widgets.HTML(HTMLButtonPrompt.format(link='./spec/', text='Download Spectra'),
        layout=Layout(
            margin='10pt'
        ))

spec_process = widgets.HTML(HTMLButtonPrompt.format(link='Calculator.ipynb', text='Process Spectra'),
        layout=Layout(
            margin='10pt'
        ))

DescriptText ="<br><p><b>Frequency</b> is the site energy center value in cm<sup>-1</sup>.</p>"\
    "<p><b>FWHM</b> is the site energy distribution width in cm<sup>-1</sup>.</p>"
ParamDescript = widgets.HTML(DescriptText)


ParamBox = widgets.Box([HeadBox] + ParamBoxList + [ParamDescript],
                      layout=Layout(flex_flow='column',
                                   border='solid 0.5pt',
                                   width=grid_total_width))

display(widgets.HBox([
    widgets.Box([widgets.HBox([load_button, param_drop, refresh_button]),
                specout,
                widgets.HBox([calc_button, progress_bar]),
                DisorderBox,
                widgets.HBox([specfilelbl, specfile, export_button]),
                spec_down,
                spec_process,
                 ], layout=Layout(
                    flex_flow='column',
                    align_items='center',
                )),
        widgets.Box([SetDefBox, 
                     widgets.Label(layout=Layout(height='15px')),
                     ParamBox], layout=Layout(flex_flow='column', align_items='flex-start'))
]))

with specout:
    %matplotlib notebook
    specfig = plt.figure(figsize=(4,3))
    specax = plt.gca()
    specax2 = specax.twiny()
    abs_line, = specax.plot(vaxis, 0.0*vaxis)
    build_axes()
